In [8]:
# ==========================================
#  ✅ UNIVERSAL PROJECT ROOT + DATA LOADER
# ==========================================

import sys
from pathlib import Path
import pandas as pd

# 1️⃣ Detect whether we are inside /notebooks, else assume project root
NB_DIR = Path.cwd()
ROOT = NB_DIR.parent if NB_DIR.name.lower() == "notebooks" else NB_DIR
sys.path.insert(0, str(ROOT))   # allow imports from src/


# 2️⃣ Define key folders
DATA = ROOT / "data"
OUT  = ROOT / "outputs"
OUT.mkdir(exist_ok=True)

# 3️⃣ Find the correct features CSV automatically
candidates = [
    OUT / "f1_features_weather.csv",
    OUT / "f1_features_weather_ergast.csv",
    OUT / "f1_features_weather_enhanced.csv",
    OUT / "f1_features_weather_with_qualifying.csv"
]

print("\n🔍 Searching for processed feature files...")
found = None
for p in candidates:
    print(f"  └─ {p.name}: {'✅ found' if p.exists() else '❌ missing'}")
    if p.exists():
        found = p

if not found:
    raise FileNotFoundError(
        "❌ No valid processed dataset found in /outputs/. "
        "Check the folder and file names in VSCode Explorer."
    )

# 4️⃣ Load dataset
base = pd.read_csv(found)
print(f"\n✅ Loaded dataset: {found.name}")
print(f"📊 Rows: {len(base):,}  |  Columns: {len(base.columns)}")
print("🔎 Column preview:", list(base.columns)[:10], "...")



🔍 Searching for processed feature files...
  └─ f1_features_weather.csv: ✅ found
  └─ f1_features_weather_ergast.csv: ❌ missing
  └─ f1_features_weather_enhanced.csv: ❌ missing
  └─ f1_features_weather_with_qualifying.csv: ❌ missing

✅ Loaded dataset: f1_features_weather.csv
📊 Rows: 11,041  |  Columns: 21
🔎 Column preview: ['raceId', 'driverId', 'bestLap_ms', 'bestLaps_s', 'drivers_name', 'year', 'round', 'circuitId', 'gp_name', 'date'] ...


In [ ]:


import pandas as pd

assert DATA.exists(), f"DATA folder not found: {DATA}"

# Keep only the columns we need (faster & clearer)
races = pd.read_csv(DATA/"races.csv", usecols=["raceId","year","round","date"])
drivers = pd.read_csv(DATA/"drivers.csv", usecols=["driverId","driverRef","dob"])

results = pd.read_csv(
    DATA/"results.csv",
    usecols=[
        "raceId","driverId","constructorId","grid",
        "position","positionOrder","rank","fastestLapSpeed"
    ]
)

# optional tables we’ll use for features shortly
pit = pd.read_csv(DATA/"pit_stops.csv", usecols=["raceId","driverId","stop"])
dstand = pd.read_csv(DATA/"driver_standings.csv", usecols=["raceId","driverId","points","position","wins"])
cstand = pd.read_csv(DATA/"constructor_standings.csv", usecols=["raceId","constructorId","points","position","wins"])

# quick sanity snapshot
def info(df, name):
    print(f"{name:>24}: shape={df.shape}  cols={list(df.columns)}")

info(races,  "races")
info(drivers,"drivers")
info(results,"results")
info(pit,    "pit_stops")
info(dstand, "driver_standings")
info(cstand, "constructor_standings")

# show a peek to confirm alignment keys exist
display(races.head(3))
display(results.head(3))
display(drivers.head(3))


                   races: shape=(1125, 4)  cols=['raceId', 'year', 'round', 'date']
                 drivers: shape=(861, 3)  cols=['driverId', 'driverRef', 'dob']
                 results: shape=(26759, 8)  cols=['raceId', 'driverId', 'constructorId', 'grid', 'position', 'positionOrder', 'rank', 'fastestLapSpeed']
               pit_stops: shape=(11371, 3)  cols=['raceId', 'driverId', 'stop']
        driver_standings: shape=(34863, 5)  cols=['raceId', 'driverId', 'points', 'position', 'wins']
   constructor_standings: shape=(13391, 5)  cols=['raceId', 'constructorId', 'points', 'position', 'wins']


,raceId,year,round,date
0,1,2009,1,2009-03-29
1,2,2009,2,2009-04-05
2,3,2009,3,2009-04-19


,raceId,driverId,constructorId,grid,position,positionOrder,rank,fastestLapSpeed
0,18,1,1,1,1,1,2,218.300
1,18,2,2,5,2,2,3,217.586
2,18,3,3,7,3,3,5,216.719


,driverId,driverRef,dob
0,1,hamilton,1985-01-07
1,2,heidfeld,1977-05-10
2,3,rosberg,1985-06-27


In [10]:
# ==========================================
# 2.3 — Create PIT STOP COUNT per race+driver
# ==========================================

# pit table already loaded as `pit` in step 2.2
# It has columns: raceId, driverId, stop (each row = 1 pit stop event)

# Group and count stops
pit_counts = (
    pit.groupby(["raceId", "driverId"])
       .size()
       .reset_index(name="pit_stops")
)

print("Pit stop counts shape:", pit_counts.shape)
display(pit_counts.head(10))


Pit stop counts shape: (5575, 3)


,raceId,driverId,pit_stops
0,841,1,2
1,841,2,2
2,841,3,1
3,841,4,3
4,841,5,1
5,841,10,1
6,841,13,3
7,841,15,2
8,841,16,2
9,841,17,3


In [11]:
# ==========================================
# 2.4 — Driver standings BEFORE the race
#       (previous round in same season)
# ==========================================

import numpy as np
import pandas as pd

# We use races (year, round, raceId) and driver_standings (raceId, driverId, points/position/wins)
assert {"raceId","year","round"}.issubset(races.columns)
assert {"raceId","driverId","points","position","wins"}.issubset(dstand.columns)

# Map (raceId -> (year, round)) and ((year, round) -> raceId)
yr_rd = races.set_index("raceId")[["year","round"]]
rd_to_race = races.set_index(["year","round"])["raceId"].to_dict()

def prev_raceid(rid: int):
    """Return the previous round's raceId for the same year, or NaN if none."""
    y, r = yr_rd.loc[rid, ["year","round"]]
    if pd.isna(y) or pd.isna(r) or int(r) <= 1:
        return np.nan
    return rd_to_race.get((int(y), int(r) - 1), np.nan)

# Compute previous raceId for every race
races_prev = races[["raceId"]].copy()
races_prev["prevRaceId"] = races_prev["raceId"].map(prev_raceid)

# Join previous driver standings (these are the standings BEFORE our current race)
dstand_prev = races_prev.merge(
    dstand, left_on="prevRaceId", right_on="raceId", how="left", suffixes=("","_stand")
)

# Rename to clear semantics and keep only what we need
dstand_prev = dstand_prev.rename(columns={
    "raceId_x" : "raceId",           # current race
    "driverId" : "driverId",
    "points"   : "driver_pts_pre",
    "position" : "driver_pos_pre",
    "wins"     : "driver_wins_season_pre"
})[["raceId","driverId","driver_pts_pre","driver_pos_pre","driver_wins_season_pre"]]

print("driver pre-race standings shape:", dstand_prev.shape)
display(dstand_prev.head(10))


driver pre-race standings shape: (31746, 5)


,raceId,driverId,driver_pts_pre,driver_pos_pre,driver_wins_season_pre
0,1,NaN,NaN,NaN,NaN
1,2,8.0,0.0,15.0,0.0
2,2,9.0,0.0,14.0,0.0
3,2,20.0,0.0,13.0,0.0
4,2,17.0,0.0,12.0,0.0
5,2,21.0,0.0,11.0,0.0
6,2,2.0,0.0,10.0,0.0
7,2,16.0,0.0,9.0,0.0
8,2,7.0,1.0,8.0,0.0
9,2,67.0,2.0,7.0,0.0


In [12]:
# ==========================================
# 2.5 — Constructor standings BEFORE the race
#       (previous round in same season)
# ==========================================

import numpy as np
import pandas as pd

# We already have `races_prev` from step 2.4 and `cstand` loaded in 2.2
# races_prev has: raceId (current), prevRaceId (previous round in same season)

assert {"raceId", "constructorId", "points", "position", "wins"}.issubset(cstand.columns)

# Join constructor standings from the PREVIOUS race (pre-race form)
cstand_prev = races_prev.merge(
    cstand,
    left_on="prevRaceId",
    right_on="raceId",
    how="left",
    suffixes=("", "_stand")
)

# Tidy and keep only what we need; rename for clarity
cstand_prev = cstand_prev.rename(columns={
    "raceId_x" : "raceId",                    # current race
    "constructorId": "constructorId",
    "points"   : "team_pts_pre",
    "position" : "team_pos_pre",
    "wins"     : "team_wins_season_pre"
})[["raceId","constructorId","team_pts_pre","team_pos_pre","team_wins_season_pre"]]

print("constructor pre-race standings shape:", cstand_prev.shape)
display(cstand_prev.head(10))


constructor pre-race standings shape: (12602, 5)


,raceId,constructorId,team_pts_pre,team_pos_pre,team_wins_season_pre
0,1,NaN,NaN,NaN,NaN
1,2,6.0,0.0,9.0,0.0
2,2,2.0,0.0,7.0,0.0
3,2,9.0,0.0,8.0,0.0
4,2,10.0,0.0,6.0,0.0
5,2,5.0,3.0,5.0,0.0
6,2,3.0,3.0,4.0,0.0
7,2,4.0,4.0,3.0,0.0
8,2,7.0,11.0,2.0,0.0
9,2,23.0,18.0,1.0,1.0


In [ ]:
# ==========================================
# 2.6 + 2.7 + 2.8 — results features + age + merge + save
# ==========================================

import pandas as pd
import numpy as np
from pathlib import Path

# We assume these are already defined from 2.2–2.5:
# ROOT, DATA, OUT, base, races, drivers, results, pit_counts, dstand_prev, cstand_prev

# ---------- 2.6a) Results-based features (grid, finish, fastest-lap) ----------
res_feats = results.copy()

# Use numeric finish position from positionOrder (safer than 'position' which may be '\N')
res_feats["finish_pos"] = pd.to_numeric(res_feats["positionOrder"], errors="coerce")

# Fastest-lap rank & average speed (can be NaN)
res_feats["fl_rank"] = pd.to_numeric(res_feats["rank"], errors="coerce")
res_feats["fl_avg_speed_kph"] = pd.to_numeric(res_feats["fastestLapSpeed"], errors="coerce")

res_feats = res_feats[["raceId","driverId","constructorId","grid","finish_pos","fl_rank","fl_avg_speed_kph"]]

# ---------- 2.6b) Driver age at race date ----------
drv_age = (
    base[["raceId","driverId"]].drop_duplicates()
    .merge(drivers[["driverId","dob"]], on="driverId", how="left")
    .merge(races[["raceId","date"]], on="raceId", how="left")
)

drv_age["dob"] = pd.to_datetime(drv_age["dob"], errors="coerce")
drv_age["date"] = pd.to_datetime(drv_age["date"], errors="coerce")
drv_age["driver_age"] = (drv_age["date"] - drv_age["dob"]).dt.days / 365.25
drv_age_small = drv_age[["raceId","driverId","driver_age"]].drop_duplicates()

# ---------- 2.7) Merge everything onto your base table ----------
enh = base.copy()

# results-based features
enh = enh.merge(res_feats, on=["raceId","driverId"], how="left")

# pit stops
if 'pit_counts' in globals():
    enh = enh.merge(pit_counts, on=["raceId","driverId"], how="left")
else:
    print("⚠️ pit_counts not found; run step 2.3 first")

# driver pre-race standings (from previous round)
if 'dstand_prev' in globals():
    enh = enh.merge(dstand_prev, on=["raceId","driverId"], how="left")
else:
    print("⚠️ dstand_prev not found; run step 2.4 first")

# constructor pre-race standings (from previous round) via constructorId
if 'cstand_prev' in globals():
    enh = enh.merge(cstand_prev, on=["raceId","constructorId"], how="left")
else:
    print("⚠️ cstand_prev not found; run step 2.5 first")

# driver age
enh = enh.merge(drv_age_small, on=["raceId","driverId"], how="left")

# ---------- 2.8) Basic hygiene + save ----------
# cast numerics where appropriate
num_like = [
    "grid","finish_pos","fl_rank","fl_avg_speed_kph",
    "pit_stops",
    "driver_pts_pre","driver_pos_pre","driver_wins_season_pre",
    "team_pts_pre","team_pos_pre","team_wins_season_pre",
    "driver_age"
]
for c in num_like:
    if c in enh.columns:
        enh[c] = pd.to_numeric(enh[c], errors="coerce")

# leave NaNs as-is (we'll impute at model time)
OUT.mkdir(exist_ok=True)
out_path = OUT / "f1_features_weather_enhanced.csv"
enh.to_csv(out_path, index=False)

display(enh.sample(5))


✅ Enhanced dataset saved to: f:\Personal Projects\F1-FastestLap-Predictor\outputs\f1_features_weather_enhanced.csv
🧮 Rows: 11041  | Columns: 34
🔎 New columns present: ['grid', 'finish_pos', 'fl_rank', 'fl_avg_speed_kph', 'pit_stops', 'driver_pts_pre', 'driver_pos_pre', 'driver_wins_season_pre', 'team_pts_pre', 'team_pos_pre', 'team_wins_season_pre', 'driver_age'] ...


,raceId,driverId,bestLap_ms,bestLaps_s,drivers_name,year,round,circuitId,gp_name,date,...,fl_rank,fl_avg_speed_kph,pit_stops,driver_pts_pre,driver_pos_pre,driver_wins_season_pre,team_pts_pre,team_pos_pre,team_wins_season_pre,driver_age
1648,83,30,81476,81.476,Michael Schumacher,2005,13,11,Hungarian Grand Prix,2005-07-31,...,3.0,193.573,NaN,47.0,3.0,1.0,78.0,3.0,1.0,36.572211
645,34,8,96483,96.483,Kimi Räikkönen,2008,17,17,Chinese Grand Prix,2008-10-19,...,2.0,203.389,NaN,63.0,4.0,2.0,142.0,1.0,7.0,29.007529
8837,1024,846,105716,105.716,Lando Norris,2019,15,15,Singapore Grand Prix,2019-09-22,...,11.0,172.412,1.0,25.0,14.0,0.0,83.0,4.0,0.0,19.857632
3797,190,55,103669,103.669,Jean Alesi,1999,16,22,Japanese Grand Prix,1999-10-31,...,NaN,NaN,NaN,1.0,16.0,0.0,4.0,8.0,0.0,35.386721
7025,930,18,91162,91.162,Jenson Button,2015,5,4,Spanish Grand Prix,2015-05-10,...,10.0,183.826,3.0,0.0,17.0,0.0,0.0,9.0,0.0,35.304586
